Import libraries

In [1]:
import os 
import copy
import numpy as np 
import pandas as pd
from tqdm import tqdm
from pprint import pprint
import matplotlib.pyplot as plt 

import rdkit.Chem as Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem

from utils import load_pickle, pickle_data

Settings

In [2]:
cache_folder = "./cache"
if not os.path.exists(cache_folder): os.makedirs(cache_folder)

Helper Functions

In [ ]:
def get_diff_expt_param_no_energy(test_info, train_dict):

    if len(train_dict) == 0 : return {}

    test_adduct = test_info["precursor_type"]
    test_instrument = test_info["instrument_type"]

    diff_adduct = 0
    diff_instrument = 0 
    diff_adduct_instrument = 0 
    reason_diff = {}

    for _, rec in train_dict.items():

        train_info = rec["train_info"]
        train_adduct = train_info["precursor_type"]
        train_instrument = train_info["instrument_type"]

        check_diff_instrument = test_instrument != train_instrument
        check_diff_adduct = test_adduct != train_adduct

        if check_diff_adduct and check_diff_instrument: diff_adduct_instrument += 1 
        elif check_diff_instrument: diff_instrument += 1 
        elif check_diff_adduct: diff_adduct += 1 

    reason_diff["diff_adduct"] = diff_adduct
    reason_diff["diff_instrument"] = diff_instrument
    reason_diff["diff_adduct_instrument"] = diff_adduct_instrument
    
    return reason_diff

In [7]:
def analyze_influence_records(test_info, train_recs, include_energy = True):

    # Get number of times this training sample is helpful 
    helpful = {k:v for k,v in train_recs.items() if v["score"] > 0}
    harmful = {k:v for k,v in train_recs.items() if v["score"] < 0}

    # Separate out to experimental settings 
    helpful_same_expt = {k:v for k,v in helpful.items() if same_expt(test_info, v, include_energy = True)}
    helpful_diff_expt = {k:v for k,v in helpful.items() if not same_expt(test_info, v, include_energy = True)}

    harmful_same_expt = {k:v for k,v in harmful.items() if same_expt(test_info, v, include_energy = True)}
    harmful_diff_expt = {k:v for k,v in harmful.items() if not same_expt(test_info, v, include_energy = True)}

    # Get the reasons for different experiments
    if include_energy:
        reason_helpful_diff_expt = get_diff_expt_param_w_energy(test_info, helpful_diff_expt)
        reason_harmful_diff_expt = get_diff_expt_param_w_energy(test_info, harmful_diff_expt)

    else: 
        reason_helpful_diff_expt = get_diff_expt_param_no_energy(test_info, helpful_diff_expt)
        reason_harmful_diff_expt = get_diff_expt_param_no_energy(test_info, harmful_diff_expt)

    # Get the distance between the helpful and harmful molecules 
    helpful_dist = compute_dist_mols(test_info, helpful)
    harmful_dist = compute_dist_mols(test_info, harmful)

    # Merge the results 
    results = {} 
    results["helpful"] = {"count": len(helpful)}
    results["harmful"] = {"count": len(harmful)}

    results["helpful"]["same_expt_count"] = len(helpful_same_expt) 
    results["helpful"]["diff_expt_count"] = len(helpful_diff_expt) 
    results["helpful"]["diff_expt_reason"] = reason_helpful_diff_expt
    results["helpful"]["dist"] = helpful_dist

    results["harmful"]["same_expt_count"] = len(harmful_same_expt) 
    results["harmful"]["diff_expt_count"] = len(harmful_diff_expt)
    results["harmful"]["diff_expt_reason"] = reason_harmful_diff_expt
    results["harmful"]["dist"] = harmful_dist

    results["same_expt"] = {"helpful": len(helpful_same_expt), 
                            "harmful": len(harmful_same_expt),
                            "count": len(helpful_same_expt) + len(harmful_same_expt)}

    results["diff_expt"] = {"helpful": len(helpful_diff_expt), 
                            "harmful": len(harmful_diff_expt),
                            "count": len(helpful_diff_expt) + len(harmful_diff_expt)}
    
    return results

In [9]:
def merge_reasons(master, sub):

    if sub == {}: return master

    for k, _ in master.items():
        if sub[k] is None: continue 
        master[k] += sub[k]

    return master 

def merge_results_wo_energy(results):

    merged = {}
    merged["diff_mol"] = {"same_expt" : {"helpful": 0,
                                         "harmful": 0,
                                         "count": 0},
                          "diff_expt" : {"helpful": 0,
                                         "harmful": 0,
                                         "count": 0},
                                         
                          "harmful" : {"diff_expt_count": 0,
                                       "same_expt_count": 0,
                                       "count": 0,
                                       "dist": 0,
                                       "diff_expt_reason": {"diff_adduct": 0,
                                                           "diff_adduct_instrument": 0,
                                                           "diff_instrument": 0}},
                          "helpful" : {"diff_expt_count": 0,
                                       "same_expt_count": 0,
                                       "count": 0,
                                       "dist": 0,
                                       "diff_expt_reason": {"diff_adduct": 0,
                                                           "diff_adduct_instrument": 0,
                                                           "diff_instrument": 0}}}
    
    merged["same_mol"] = {"same_expt" : {"helpful": 0,
                                         "harmful": 0,
                                         "count": 0},
                          "diff_expt" : {"helpful": 0,
                                         "harmful": 0,
                                         "count": 0},
                          "harmful" : {"diff_expt_count": 0,
                                       "same_expt_count": 0,
                                       "count": 0,
                                       "diff_expt_reason": {"diff_adduct": 0,
                                                           "diff_adduct_instrument": 0,
                                                           "diff_instrument": 0}},
                          "helpful" : {"diff_expt_count": 0,
                                       "same_expt_count": 0,
                                       "count": 0,
                                       "diff_expt_reason": {"diff_adduct": 0,
                                                           "diff_adduct_instrument": 0,
                                                           "diff_instrument": 0}}}

    for _, r in results.items():

        # For different mols
        merged["diff_mol"]["same_expt"]["helpful"] += r["diff_mol"]["same_expt"]["helpful"] 
        merged["diff_mol"]["same_expt"]["harmful"] += r["diff_mol"]["same_expt"]["harmful"] 
        merged["diff_mol"]["same_expt"]["count"] += r["diff_mol"]["same_expt"]["count"] 

        merged["diff_mol"]["diff_expt"]["helpful"] += r["diff_mol"]["diff_expt"]["helpful"] 
        merged["diff_mol"]["diff_expt"]["harmful"] += r["diff_mol"]["diff_expt"]["harmful"] 
        merged["diff_mol"]["diff_expt"]["count"] += r["diff_mol"]["diff_expt"]["count"]   

        merged["diff_mol"]["harmful"]["diff_expt_count"] += r["diff_mol"]["harmful"]["diff_expt_count"] 
        merged["diff_mol"]["harmful"]["same_expt_count"] += r["diff_mol"]["harmful"]["same_expt_count"] 
        merged["diff_mol"]["harmful"]["count"] += r["diff_mol"]["harmful"]["count"]   
        merged["diff_mol"]["harmful"]["dist"] += r["diff_mol"]["harmful"]["dist"]   
        merged["diff_mol"]["harmful"]["diff_expt_reason"] = merge_reasons(merged["diff_mol"]["harmful"]["diff_expt_reason"], r["diff_mol"]["harmful"]["diff_expt_reason"]) 

        merged["diff_mol"]["helpful"]["diff_expt_count"] += r["diff_mol"]["helpful"]["diff_expt_count"] 
        merged["diff_mol"]["helpful"]["same_expt_count"] += r["diff_mol"]["helpful"]["same_expt_count"] 
        merged["diff_mol"]["helpful"]["count"] += r["diff_mol"]["helpful"]["count"]   
        merged["diff_mol"]["helpful"]["dist"] += r["diff_mol"]["helpful"]["dist"]   
        merged["diff_mol"]["helpful"]["diff_expt_reason"] = merge_reasons(merged["diff_mol"]["helpful"]["diff_expt_reason"], r["diff_mol"]["helpful"]["diff_expt_reason"]) 

        # For same mols
        merged["same_mol"]["same_expt"]["helpful"] += r["same_mol"]["same_expt"]["helpful"] 
        merged["same_mol"]["same_expt"]["harmful"] += r["same_mol"]["same_expt"]["harmful"] 
        merged["same_mol"]["same_expt"]["count"] += r["same_mol"]["same_expt"]["count"] 

        merged["same_mol"]["diff_expt"]["helpful"] += r["same_mol"]["diff_expt"]["helpful"] 
        merged["same_mol"]["diff_expt"]["harmful"] += r["same_mol"]["diff_expt"]["harmful"] 
        merged["same_mol"]["diff_expt"]["count"] += r["same_mol"]["diff_expt"]["count"] 

        merged["same_mol"]["harmful"]["diff_expt_count"] += r["same_mol"]["harmful"]["diff_expt_count"] 
        merged["same_mol"]["harmful"]["same_expt_count"] += r["same_mol"]["harmful"]["same_expt_count"] 
        merged["same_mol"]["harmful"]["count"] += r["same_mol"]["harmful"]["count"]   
        merged["same_mol"]["harmful"]["diff_expt_reason"] = merge_reasons(merged["same_mol"]["harmful"]["diff_expt_reason"], r["same_mol"]["harmful"]["diff_expt_reason"]) 

        merged["same_mol"]["helpful"]["diff_expt_count"] += r["same_mol"]["helpful"]["diff_expt_count"] 
        merged["same_mol"]["helpful"]["same_expt_count"] += r["same_mol"]["helpful"]["same_expt_count"] 
        merged["same_mol"]["helpful"]["count"] += r["same_mol"]["helpful"]["count"]   
        merged["same_mol"]["helpful"]["diff_expt_reason"] = merge_reasons(merged["same_mol"]["helpful"]["diff_expt_reason"], r["same_mol"]["helpful"]["diff_expt_reason"]) 

    return merged

def merge_results_w_energy(results):

    merged = {}
    merged["diff_mol"] = {"same_expt" : {"helpful": 0,
                                         "harmful": 0,
                                         "count": 0},
                          "diff_expt" : {"helpful": 0,
                                         "harmful": 0,
                                         "count": 0},
                                         
                          "harmful" : {"diff_expt_count": 0,
                                       "same_expt_count": 0,
                                       "count": 0,
                                       "dist": 0,
                                       "diff_expt_reason": {"total_diff_CE": 0,
                                                           "diff_CE": 0,
                                                           "diff_adduct": 0,
                                                           "diff_adduct_CE": 0,
                                                           "diff_adduct_instrument": 0,
                                                           "diff_adduct_instrument_CE": 0,
                                                           "diff_instrument": 0,
                                                           "diff_instrument_CE": 0}},
                          "helpful" : {"diff_expt_count": 0,
                                       "same_expt_count": 0,
                                       "count": 0,
                                       "dist": 0,
                                       "diff_expt_reason": {"total_diff_CE": 0,
                                                           "diff_CE": 0,
                                                           "diff_adduct": 0,
                                                           "diff_adduct_CE": 0,
                                                           "diff_adduct_instrument": 0,
                                                           "diff_adduct_instrument_CE": 0,
                                                           "diff_instrument": 0,
                                                           "diff_instrument_CE": 0}}}
    
    merged["same_mol"] = {"same_expt" : {"helpful": 0,
                                         "harmful": 0,
                                         "count": 0},
                          "diff_expt" : {"helpful": 0,
                                         "harmful": 0,
                                         "count": 0},
                          "harmful" : {"diff_expt_count": 0,
                                       "same_expt_count": 0,
                                       "count": 0,
                                       "diff_expt_reason": {"total_diff_CE":0,
                                                           "diff_CE": 0,
                                                           "diff_adduct": 0,
                                                           "diff_adduct_CE": 0,
                                                           "diff_adduct_instrument": 0,
                                                           "diff_adduct_instrument_CE": 0,
                                                           "diff_instrument": 0,
                                                           "diff_instrument_CE": 0}},
                          "helpful" : {"diff_expt_count": 0,
                                       "same_expt_count": 0,
                                       "count": 0,
                                       "diff_expt_reason": {"total_diff_CE": 0,
                                                           "diff_CE": 0,
                                                           "diff_adduct": 0,
                                                           "diff_adduct_CE": 0,
                                                           "diff_adduct_instrument": 0,
                                                           "diff_adduct_instrument_CE": 0,
                                                           "diff_instrument": 0,
                                                           "diff_instrument_CE": 0}}}

    for _, r in results.items():

        # For different mols
        merged["diff_mol"]["same_expt"]["helpful"] += r["diff_mol"]["same_expt"]["helpful"] 
        merged["diff_mol"]["same_expt"]["harmful"] += r["diff_mol"]["same_expt"]["harmful"] 
        merged["diff_mol"]["same_expt"]["count"] += r["diff_mol"]["same_expt"]["count"] 

        merged["diff_mol"]["diff_expt"]["helpful"] += r["diff_mol"]["diff_expt"]["helpful"] 
        merged["diff_mol"]["diff_expt"]["harmful"] += r["diff_mol"]["diff_expt"]["harmful"] 
        merged["diff_mol"]["diff_expt"]["count"] += r["diff_mol"]["diff_expt"]["count"]   

        merged["diff_mol"]["harmful"]["diff_expt_count"] += r["diff_mol"]["harmful"]["diff_expt_count"] 
        merged["diff_mol"]["harmful"]["same_expt_count"] += r["diff_mol"]["harmful"]["same_expt_count"] 
        merged["diff_mol"]["harmful"]["count"] += r["diff_mol"]["harmful"]["count"]   
        merged["diff_mol"]["harmful"]["dist"] += r["diff_mol"]["harmful"]["dist"]   
        merged["diff_mol"]["harmful"]["diff_expt_reason"] = merge_reasons(merged["diff_mol"]["harmful"]["diff_expt_reason"], r["diff_mol"]["harmful"]["diff_expt_reason"]) 

        merged["diff_mol"]["helpful"]["diff_expt_count"] += r["diff_mol"]["helpful"]["diff_expt_count"] 
        merged["diff_mol"]["helpful"]["same_expt_count"] += r["diff_mol"]["helpful"]["same_expt_count"] 
        merged["diff_mol"]["helpful"]["count"] += r["diff_mol"]["helpful"]["count"]   
        merged["diff_mol"]["helpful"]["dist"] += r["diff_mol"]["helpful"]["dist"]   
        merged["diff_mol"]["helpful"]["diff_expt_reason"] = merge_reasons(merged["diff_mol"]["helpful"]["diff_expt_reason"], r["diff_mol"]["helpful"]["diff_expt_reason"]) 

        # For same mols
        merged["same_mol"]["same_expt"]["helpful"] += r["same_mol"]["same_expt"]["helpful"] 
        merged["same_mol"]["same_expt"]["harmful"] += r["same_mol"]["same_expt"]["harmful"] 
        merged["same_mol"]["same_expt"]["count"] += r["same_mol"]["same_expt"]["count"] 

        merged["same_mol"]["diff_expt"]["helpful"] += r["same_mol"]["diff_expt"]["helpful"] 
        merged["same_mol"]["diff_expt"]["harmful"] += r["same_mol"]["diff_expt"]["harmful"] 
        merged["same_mol"]["diff_expt"]["count"] += r["same_mol"]["diff_expt"]["count"] 

        merged["same_mol"]["harmful"]["diff_expt_count"] += r["same_mol"]["harmful"]["diff_expt_count"] 
        merged["same_mol"]["harmful"]["same_expt_count"] += r["same_mol"]["harmful"]["same_expt_count"] 
        merged["same_mol"]["harmful"]["count"] += r["same_mol"]["harmful"]["count"]   
        merged["same_mol"]["harmful"]["diff_expt_reason"] = merge_reasons(merged["same_mol"]["harmful"]["diff_expt_reason"], r["same_mol"]["harmful"]["diff_expt_reason"]) 

        merged["same_mol"]["helpful"]["diff_expt_count"] += r["same_mol"]["helpful"]["diff_expt_count"] 
        merged["same_mol"]["helpful"]["same_expt_count"] += r["same_mol"]["helpful"]["same_expt_count"] 
        merged["same_mol"]["helpful"]["count"] += r["same_mol"]["helpful"]["count"]   
        merged["same_mol"]["helpful"]["diff_expt_reason"] = merge_reasons(merged["same_mol"]["helpful"]["diff_expt_reason"], r["same_mol"]["helpful"]["diff_expt_reason"]) 

    return merged

def merge_results(results, include_energy):
    if include_energy: return merge_results_w_energy(results)
    return merge_results_wo_energy(results)

In [10]:
def get_average_reasons_w_energy(results):

    results = copy.deepcopy(results)
    new_reasons = {}
    n_expt_diff_CE = results["diff_expt_reason"]["diff_CE"] + \
                     results["diff_expt_reason"]["diff_adduct_CE"] +\
                     results["diff_expt_reason"]["diff_adduct_instrument_CE"] +\
                     results["diff_expt_reason"]["diff_instrument_CE"]

    new_reasons["avg_CE"]= round(results["diff_expt_reason"]["total_diff_CE"] / n_expt_diff_CE, 3) 
    new_reasons["percent_diff_CE"]= round(results["diff_expt_reason"]["diff_CE"] / (1  + results["diff_expt_count"]) * 100, 3) 
    new_reasons["percent_diff_adduct"]= round(results["diff_expt_reason"]["diff_adduct"] / (1  + results["diff_expt_count"]) * 100, 3) 
    new_reasons["percent_diff_adduct_CE"]= round(results["diff_expt_reason"]["diff_adduct_CE"] / (1  + results["diff_expt_count"])* 100, 3) 
    new_reasons["percent_diff_adduct_instrument"]= round(results["diff_expt_reason"]["diff_adduct_instrument"] / (1  + results["diff_expt_count"])* 100, 3) 
    new_reasons["percent_diff_adduct_instrument_CE"]= round(results["diff_expt_reason"]["diff_adduct_instrument_CE"] / (1  + results["diff_expt_count"])* 100, 3) 
    new_reasons["percent_diff_instrument"]= round(results["diff_expt_reason"]["diff_instrument"] / (1  + results["diff_expt_count"]) * 100, 3) 
    new_reasons["percent_diff_instrument_CE"]= round(results["diff_expt_reason"]["diff_instrument_CE"] / (1  + results["diff_expt_count"]) * 100, 3) 

    results["diff_expt_reason"] = new_reasons

    return results 

def get_average_reasons_wo_energy(results):

    results = copy.deepcopy(results)
    new_reasons = {}

    new_reasons["percent_diff_adduct"]= round(results["diff_expt_reason"]["diff_adduct"] / (1  + results["diff_expt_count"]) * 100, 3) 
    new_reasons["percent_diff_adduct_instrument"]= round(results["diff_expt_reason"]["diff_adduct_instrument"] / (1  + results["diff_expt_count"]) * 100, 3) 
    new_reasons["percent_diff_instrument"]= round(results["diff_expt_reason"]["diff_instrument"] /(1  + results["diff_expt_count"]) * 100, 3) 

    results["diff_expt_reason"] = new_reasons

    return results 

def get_average_reasons(results, include_energy = True):

    if include_energy: return get_average_reasons_w_energy(results)
    
    return get_average_reasons_wo_energy(results)

def get_average(merged_results, include_energy = True):
    
    # Diff mol
    merged_results["diff_mol"]["diff_expt"]["percent_harmful"] = round(merged_results["diff_mol"]["diff_expt"]["harmful"] / merged_results["diff_mol"]["diff_expt"]["count"] * 100, 3) 
    merged_results["diff_mol"]["diff_expt"]["percent_helpful"] = round(merged_results["diff_mol"]["diff_expt"]["helpful"] / merged_results["diff_mol"]["diff_expt"]["count"] * 100, 3) 

    merged_results["diff_mol"]["same_expt"]["percent_harmful"] = round(merged_results["diff_mol"]["same_expt"]["harmful"] / merged_results["diff_mol"]["same_expt"]["count"] * 100, 3) 
    merged_results["diff_mol"]["same_expt"]["percent_helpful"] = round(merged_results["diff_mol"]["same_expt"]["helpful"] / merged_results["diff_mol"]["same_expt"]["count"] * 100, 3) 
    
    merged_results["diff_mol"]["harmful"]["percent"] = round(merged_results["diff_mol"]["harmful"]["count"] / (merged_results["diff_mol"]["harmful"]["count"] + merged_results["diff_mol"]["helpful"]["count"] ) * 100, 3) 
    merged_results["diff_mol"]["helpful"]["percent"] = round(merged_results["diff_mol"]["helpful"]["count"] / (merged_results["diff_mol"]["harmful"]["count"] + merged_results["diff_mol"]["helpful"]["count"] ) * 100, 3) 
    
    merged_results["diff_mol"]["harmful"]["percent_same_expt"] = round(merged_results["diff_mol"]["harmful"]["same_expt_count"] / (merged_results["diff_mol"]["harmful"]["same_expt_count"] + merged_results["diff_mol"]["helpful"]["diff_expt_count"] ) * 100, 3) 
    merged_results["diff_mol"]["harmful"]["percent_diff_expt"] = round(merged_results["diff_mol"]["harmful"]["diff_expt_count"] / (merged_results["diff_mol"]["harmful"]["same_expt_count"] + merged_results["diff_mol"]["helpful"]["diff_expt_count"] ) * 100, 3) 
    merged_results["diff_mol"]["helpful"]["percent_same_expt"] = round(merged_results["diff_mol"]["helpful"]["same_expt_count"] / (merged_results["diff_mol"]["helpful"]["same_expt_count"] + merged_results["diff_mol"]["helpful"]["diff_expt_count"] ) * 100, 3) 
    merged_results["diff_mol"]["helpful"]["percent_diff_expt"] = round(merged_results["diff_mol"]["helpful"]["diff_expt_count"] / (merged_results["diff_mol"]["helpful"]["same_expt_count"] + merged_results["diff_mol"]["helpful"]["diff_expt_count"] ) * 100, 3) 

    merged_results["diff_mol"]["harmful"]["avg_dist"] = round(merged_results["diff_mol"]["harmful"]["dist"]  / merged_results["diff_mol"]["harmful"]["count"], 3) 
    merged_results["diff_mol"]["helpful"]["avg_dist"] = round(merged_results["diff_mol"]["helpful"]["dist"]  / merged_results["diff_mol"]["helpful"]["count"], 3) 
    
    # Same mol
    merged_results["same_mol"]["harmful"]["percent"] = round(merged_results["same_mol"]["harmful"]["count"] / (merged_results["same_mol"]["harmful"]["count"] + merged_results["same_mol"]["helpful"]["count"] ) * 100, 3) 
    merged_results["same_mol"]["helpful"]["percent"] = round(merged_results["same_mol"]["helpful"]["count"] / (merged_results["same_mol"]["harmful"]["count"] + merged_results["same_mol"]["helpful"]["count"] ) * 100, 3) 
    # Diff expt
    merged_results["diff_expt"] = {}
    merged_results["diff_expt"]["count"] = merged_results["same_mol"]["diff_expt"]["count"] + merged_results["diff_mol"]["diff_expt"]["count"]
    merged_results["diff_expt"]["harmful"] = merged_results["same_mol"]["diff_expt"]["harmful"] + merged_results["diff_mol"]["diff_expt"]["harmful"]
    merged_results["diff_expt"]["helpful"] = merged_results["same_mol"]["diff_expt"]["helpful"] + merged_results["diff_mol"]["diff_expt"]["helpful"]

    merged_results["diff_expt"]["percent_harmful"] = round(merged_results["diff_expt"]["harmful"] / merged_results["diff_expt"]["count"] * 100,3)
    merged_results["diff_expt"]["percent_helpful"] = round(merged_results["diff_expt"]["helpful"] / merged_results["diff_expt"]["count"] * 100,3)

    # Same expt
    merged_results["same_expt"] = {}
    merged_results["same_expt"]["count"] = merged_results["same_mol"]["same_expt"]["count"] + merged_results["diff_mol"]["same_expt"]["count"]
    merged_results["same_expt"]["harmful"] = merged_results["same_mol"]["same_expt"]["harmful"] + merged_results["diff_mol"]["same_expt"]["harmful"]
    merged_results["same_expt"]["helpful"] = merged_results["same_mol"]["same_expt"]["helpful"] + merged_results["diff_mol"]["same_expt"]["helpful"]

    merged_results["same_expt"]["percent_harmful"] = round(merged_results["same_expt"]["harmful"] / merged_results["same_expt"]["count"] * 100,3)
    merged_results["same_expt"]["percent_helpful"] = round(merged_results["same_expt"]["helpful"] / merged_results["same_expt"]["count"] * 100,3)

    merged_results["diff_mol"]["harmful"]["diff_expt_reason"] = get_average_reasons(merged_results["diff_mol"]["harmful"], include_energy = include_energy)
    merged_results["diff_mol"]["helpful"]["diff_expt_reason"] = get_average_reasons(merged_results["diff_mol"]["helpful"], include_energy = include_energy)

    merged_results["same_mol"]["harmful"]["diff_expt_reason"] = get_average_reasons(merged_results["same_mol"]["harmful"], include_energy = include_energy)
    merged_results["same_mol"]["helpful"]["diff_expt_reason"] = get_average_reasons(merged_results["same_mol"]["helpful"], include_energy = include_energy)

    return merged_results

Get information on the random results

In [ ]:
folders = ["../FP_prediction/mist/models_cached/morgan4_4096", "../FP_prediction/baseline_models/models_cached/morgan4_4096"]
all_info_random_path = os.path.join(cache_folder, "all_info_random.pkl")

for folder in folders:
    
    
    folder = Path(folder)

    for dataset in ["canopus", "massspecgym"]:
        
        if dataset == "canopus": include_energy = False
        else: include_energy = True 
            
            if dataset == "canopus" and "mist" not in folder: inchikey_key = "inchikey_original"
            else: inchikey_key = "inchikey"

            current_folder = os.path.join(folder, dataset)
            if dataset not in all_info_random: all_info_random[dataset] = {}

            for model in tqdm(os.listdir(current_folder)):

                # Analyze for random only 
                if "random" not in model: continue

                model_folder = os.path.join(current_folder, model)
                influence_scores_path = os.path.join(model_folder, "EK-FAC_scores_w_info.pkl")
                if not os.path.exists(influence_scores_path): continue
                influence_scores = load_pickle(influence_scores_path)
                merged_results = {}
                
                for test_id, results in tqdm(influence_scores.items()):
                    
                    test_info = list(results.values())[0]["test_info"]
                    test_mol = test_info["inchikey"]

                    train_same_mol = {train_id: v for train_id, v in results.items() if test_mol[:14] == v["train_info"][inchikey_key][:14]}
                    train_diff_mol = {train_id: v for train_id, v in results.items() if test_mol[:14] != v["train_info"][inchikey_key][:14]}

                    same_mol_analysis = analyze_influence_records(test_info, train_same_mol, include_energy = include_energy)
                    diff_mol_analysis = analyze_influence_records(test_info, train_diff_mol, include_energy = include_energy)

                    merged_results[test_id] = {"same_mol": same_mol_analysis,
                                            "diff_mol": diff_mol_analysis}
                    
                merged_results = merge_results(merged_results, include_energy = include_energy)        
                merged_results = get_average(merged_results, include_energy = include_energy)
                all_info_random[dataset][model] = merged_results

    pickle_data(all_info_random, all_info_random_path)

all_info_random = load_pickle(all_info_random_path)

Get percentage of positive vs negative influence scores

In [ ]:
folders = ["../FP_prediction/baseline_models/models_cached/morgan4_4096", ]
percent_scores_path = os.path.join(cache_folder, "percent_scores.pkl")

if not os.path.exists(percent_scores_path):

    all_percent_scores = {}

    for folder in folders:
        
        for dataset in tqdm(["massspecgym", "canopus"]):
            
            if dataset not in all_percent_scores: all_percent_scores[dataset] = {} 

            current_folder = os.path.join(folder, dataset)

            for model in tqdm(os.listdir(current_folder)):

                model_folder = os.path.join(current_folder, model)
            
                train_ids_path = os.path.join(model_folder, "train_ids.pkl")
                test_ids_path = os.path.join(model_folder, "test_ids.pkl")
                influence_scores_path = os.path.join(model_folder, "EK-FAC_scores.pkl")

                train_ids = load_pickle(train_ids_path)
                test_ids = load_pickle(test_ids_path)
                influence_scores = load_pickle(influence_scores_path)

                percent_pos = round((influence_scores["all_modules"] > 0).float().mean(-1).mean().item() * 100, 3)
                percent_neg = round((influence_scores["all_modules"] < 0).float().mean(-1).mean().item() * 100, 3)

                all_percent_scores[dataset][model] = {"pos": percent_pos, "neg": percent_neg}
    
    pickle_data(all_percent_scores, percent_scores_path)

else:
    all_percent_scores = load_pickle(percent_scores_path)

pprint(all_percent_scores)

Plot percentage of negative training examples

In [ ]:
for dataset in ["massspecgym", "canopus"]:

    plt.figure(figsize=(8, 6))

    models = ["binned_4096", "MS_4096", "formula_4096"]
    splits = ["scaffold", "inchikey", "random"]

    bar_width = 0.23
    random, scaffold, inchikey = [],[],[]

    for model in models:
        random.extend([s["neg"] for k, s in all_percent_scores[dataset].items() if model in k and "random" in k])
        scaffold.extend([s["neg"] for k, s in all_percent_scores[dataset].items() if model in k and "scaffold" in k])   
        inchikey.extend([s["neg"] for k, s in all_percent_scores[dataset].items() if model in k and "inchikey" in k])    

    random_bar = np.arange(len(models)) 
    scaffold_bar = [x + bar_width + 0.01 for x in random_bar] 
    inchikey_bar = [x + bar_width + 0.01 for x in scaffold_bar] 

    plt.bar(random_bar, random, width = bar_width, label ='random', color = "#006d77") 
    plt.bar(scaffold_bar, scaffold, width = bar_width, label ='scaffold', color = "#83c5be") 
    plt.bar(inchikey_bar, inchikey, width = bar_width, label ='inchikey', color = "#e94f37") 

    for i in range(len(random)):
        plt.text(random_bar[i], random[i], str(round(random[i], 1)) + "%", ha='center', va='bottom')
        plt.text(scaffold_bar[i] + 0.01, scaffold[i], str(round(scaffold[i], 1)) + "%", ha='center', va='bottom')
        plt.text(inchikey_bar[i] + 0.01, inchikey[i], str(round(inchikey[i], 1)) + "%", ha='center', va='bottom')

    if dataset == "canopus":
        plt.title(f"Canopus", fontsize = 14)
    if dataset == "massspecgym":
        plt.title(f"MassSpecGym", fontsize = 14)
    
    plt.xlabel('Models', fontsize = 12)
    plt.ylabel('Train samples with negative influence score (%)', fontsize = 12)
    plt.xticks([r + bar_width for r in range(len(models))], ['Binned MLP', 'MS Transformer', 'Formula Transformer'])
    plt.legend()
    plt.show()
    

Look at distribution of harmful vs helpful for identical molecules

In [ ]:
for dataset in ["massspecgym", "canopus"]:

    fig, ax = plt.subplots(layout='constrained', figsize=(8, 6))
    bar_width = 0.23
    helpful, harmful = [],[]

    models = ["binned_4096", "MS_4096", "formula_4096", "mist_4096"]

    for model in models:

        helpful.extend([s["same_mol"]["helpful"]["percent"] for k, s in all_info_random[dataset].items() if model in k])
        harmful.extend([s["same_mol"]["harmful"]["percent"] for k, s in all_info_random[dataset].items() if model in k])

        assert helpful[-1] + harmful[-1] == 100
        
    helpful_bar = np.arange(len(helpful))
    harmful_bar = [x + bar_width + 0.01 for x in helpful_bar] 
    
    # plot bars
    plt.bar(helpful_bar, helpful, width = bar_width, label = "Helpful", color = "#D3D3D3")
    plt.bar(harmful_bar, harmful, width = bar_width, label = "Harmful", color = "#F6BD60") 

    if dataset == "canopus":
        plt.title(f"Canopus", fontsize = 14)
    if dataset == "massspecgym":
        plt.title(f"MassSpecGym", fontsize = 14)
    
    plt.xlabel('Models', fontsize = 12)
    plt.ylabel('Effect of identical molecules on test samples', fontsize = 12)
    plt.ylim(bottom = 0, top = 100)
    plt.xticks([r + bar_width /2 for r in range(len(models))], ['Binned MLP', 'MS Transformer', 'Formula Transformer', "MIST"])

    for i in range(len(helpful)):
        plt.text(helpful_bar[i], helpful[i], str(round(helpful[i], 1)) + "%", ha='center', va='bottom')
        plt.text(harmful_bar[i], harmful[i], str(round(harmful[i], 1)) + "%", ha='center', va='bottom')

    # pos = []
    # for r in range(len(harmful_bar)):
    #     pos.extend([r + bar_width / 10, r + bar_width])

    # plt.xticks(pos, ["Harmful", "Helpful"] * len(harmful_bar), fontsize = 8)

    # # plt.xlabel(" "  * 15 + "Binned MLP" + " " * 45 + "MS Transformer" + " " * 30 + "Formula Transformer")
    # # plt.xticks([r + bar_width/2 for r in range(len(helpful_bar))], ['Binned MLP', 'MS Transformer', 'Formula Transformer'])
    # ax2 = ax.secondary_xaxis(-0.05)
    # ax2.set_xticks([])
    # ax2.set_xticks([r + bar_width/2 for r in range(len(helpful_bar))], ['Binned MLP', 'MS Transformer', 'Formula Transformer'])

    plt.legend()

In [ ]:
for dataset in ["massspecgym", "canopus"]:

    fig, ax = plt.subplots(layout='constrained', figsize=(8, 6))
    bar_width = 0.23
    helpful, harmful = [],[]

    models = ["binned_4096", "MS_4096", "formula_4096", "mist_4096"]

    for model in models:

        helpful.extend([s["diff_mol"]["helpful"]["avg_dist"] for k, s in all_info_random[dataset].items() if model in k])
        harmful.extend([s["diff_mol"]["harmful"]["avg_dist"] for k, s in all_info_random[dataset].items() if model in k])

    helpful_bar = np.arange(len(helpful))
    harmful_bar = [x + bar_width + 0.01 for x in helpful_bar] 
    
    # plot bars
    plt.bar(helpful_bar, helpful, width = bar_width, label = "Helpful", color = "#D3D3D3")
    plt.bar(harmful_bar, harmful, width = bar_width, label = "Harmful", color = "#F6BD60") 

    if dataset == "canopus":
        plt.title(f"Canopus", fontsize = 14)
    if dataset == "massspecgym":
        plt.title(f"MassSpecGym", fontsize = 14)
    
    plt.xlabel('Models', fontsize = 12)
    plt.ylabel('Tanimoto similarity of training molecules', fontsize = 12)
    plt.ylim(bottom = 0, top = 0.3)

    # plt.ylim(bottom = 0, top = 100)
    plt.xticks([r + bar_width /2 for r in range(len(models))], ['Binned MLP', 'MS Transformer', 'Formula Transformer', "MIST"])

    for i in range(len(helpful)):
        plt.text(helpful_bar[i], helpful[i], str(round(helpful[i], 3)), ha='center', va='bottom')
        plt.text(harmful_bar[i], harmful[i], str(round(harmful[i], 3)), ha='center', va='bottom')

    # pos = []
    # for r in range(len(harmful_bar)):
    #     pos.extend([r + bar_width / 10, r + bar_width])

    # plt.xticks(pos, ["Harmful", "Helpful"] * len(harmful_bar), fontsize = 8)

    # # plt.xlabel(" "  * 15 + "Binned MLP" + " " * 45 + "MS Transformer" + " " * 30 + "Formula Transformer")
    # # plt.xticks([r + bar_width/2 for r in range(len(helpful_bar))], ['Binned MLP', 'MS Transformer', 'Formula Transformer'])
    # ax2 = ax.secondary_xaxis(-0.05)
    # ax2.set_xticks([])
    # ax2.set_xticks([r + bar_width/2 for r in range(len(helpful_bar))], ['Binned MLP', 'MS Transformer', 'Formula Transformer'])

    plt.legend()

In [ ]:
models = ["binned_4096", "MS_4096", "formula_4096", "mist_4096"]
datasets = ["canopus", "massspecgym"]

for dataset in datasets:

    for model in models:

        harmful_identical = [s["same_mol"]["same_expt"]["harmful"] for k, s in all_info_random[dataset].items() if model in k][0]
        helpful_identical = [s["same_mol"]["same_expt"]["helpful"] for k, s in all_info_random[dataset].items() if model in k][0]
        total = harmful_identical + helpful_identical

        print(dataset, model, round(harmful_identical / (total) * 100, 1), harmful_identical, total)


Helpful and harmful, distribution of experiments

- do we have the same molecule that can contribute both negatively and positively to the test set

In [ ]:
for dataset, results in all_info_random.items():

    fig, ax = plt.subplots(layout='constrained', figsize=(8, 6))
    bar_width = 0.23
    helpful, harmful = [],[]

    models = ["binned_4096", "MS_4096", "formula_4096", "mist_4096"]

    for model in models:

        helpful.extend([s["diff_mol"]["helpful"]["percent"] for k, s in all_info_random[dataset].items() if model in k])
        harmful.extend([s["diff_mol"]["harmful"]["percent"] for k, s in all_info_random[dataset].items() if model in k])

    helpful_bar = np.arange(len(helpful))
    harmful_bar = [x + bar_width + 0.01 for x in helpful_bar] 
    
    # plot bars
    plt.bar(helpful_bar, helpful, width = bar_width, label = "Helpful", color = "#D3D3D3")
    plt.bar(harmful_bar, harmful, width = bar_width, label = "Harmful", color = "#F6BD60") 

    if dataset == "canopus":
        plt.title(f"Canopus", fontsize = 14)
    if dataset == "massspecgym":
        plt.title(f"MassSpecGym", fontsize = 14)
    
    plt.xlabel('Models', fontsize = 12)
    plt.ylabel('Tanimoto similarity of training molecules', fontsize = 12)
    plt.ylim(bottom = 0, top = 65)

    # plt.ylim(bottom = 0, top = 100)
    plt.xticks([r + bar_width /2 for r in range(len(models))], ['Binned MLP', 'MS Transformer', 'Formula Transformer', "MIST"])

    for i in range(len(helpful)):
        plt.text(helpful_bar[i], helpful[i], str(round(helpful[i], 1)) + "%", ha='center', va='bottom', fontsize = 8)
        plt.text(harmful_bar[i], harmful[i], str(round(harmful[i], 1)) + "%", ha='center', va='bottom', fontsize = 8)
    
    plt.legend()

In [ ]:
dataset = "massspecgym"

for reason in ["percent_diff_CE", "percent_diff_adduct", "percent_diff_instrument",  "percent_diff_adduct_CE", "percent_diff_adduct_instrument", "percent_diff_instrument_CE", "percent_diff_adduct_instrument_CE"]:

    reason_mapping = {'percent_diff_CE': "Collision Energy",
                        'percent_diff_adduct': "Adduct",
                        'percent_diff_adduct_CE': "Collision Energy + Adduct",
                        'percent_diff_adduct_instrument': "Instrument + Adduct",
                        'percent_diff_adduct_instrument_CE': "Instrument + Adduct + CE",
                        'percent_diff_instrument': "Instrument",
                        'percent_diff_instrument_CE':  "Instrument + CE"}

    reason_key = reason_mapping[reason]
    fig, ax = plt.subplots(layout='constrained', figsize=(8, 6))
    bar_width = 0.23
    harmful = []
    helpful = []

    for model in ["MSG_binned_4096_random", "MSG_MS_4096_random", "MSG_formula_4096_random", "MSG_mist_4096_random"]:
        
        helpful.append(all_info_random[dataset][model]["diff_mol"]["helpful"]["diff_expt_reason"]["diff_expt_reason"][reason])
        harmful.append(all_info_random[dataset][model]["diff_mol"]["harmful"]["diff_expt_reason"]["diff_expt_reason"][reason])

    helpful_bar = np.arange(len(helpful))
    harmful_bar = [x + bar_width + 0.01 for x in helpful_bar] 

    # plot bars
    plt.bar(helpful_bar, helpful, width = bar_width, label = "Helpful", color = "#D3D3D3")
    plt.bar(harmful_bar, harmful, width = bar_width, label = "Harmful", color = "#F6BD60")

    if dataset == "canopus":
        plt.title(f"Canopus", fontsize = 14)
    if dataset == "massspecgym":
        plt.title(f"{reason_key} (MassSpecGym)", fontsize = 14)

    plt.xlabel('Models', fontsize = 12)
    plt.ylabel('Percentage train samples (%)', fontsize = 12)
    plt.xticks([r + bar_width /2 for r in range(len(models))], ['Binned MLP', 'MS Transformer', 'Formula Transformer', "MIST"])

    for i in range(len(helpful)):
        plt.text(helpful_bar[i], helpful[i], str(round(helpful[i], 1)) + "%", ha='center', va='bottom', fontsize = 8)
        plt.text(harmful_bar[i], harmful[i], str(round(harmful[i], 1)) + "%", ha='center', va='bottom', fontsize = 8)

    plt.legend()

In [ ]:
dataset = "massspecgym"

fig, ax = plt.subplots(layout='constrained', figsize=(8, 6))
bar_width = 0.08
diff_CE, diff_adduct, diff_adduct_CE, diff_adduct_instrument, diff_adduct_instrument_CE, diff_instrument, diff_instrument_CE = [], [], [], [], [], [], []
diff_CE, diff_adduct, diff_adduct_CE, diff_adduct_instrument, diff_adduct_instrument_CE, diff_instrument, diff_instrument_CE = [], [], [], [], [], [], []

for model in ["MSG_binned_4096_random", "MSG_MS_4096_random", "MSG_formula_4096_random", "MSG_mist_4096_random"]:

    diff_CE.append(all_info_random[dataset][model]["diff_mol"]["harmful"]["diff_expt_reason"]["diff_expt_reason"]["percent_diff_CE"])
    diff_adduct.append(all_info_random[dataset][model]["diff_mol"]["harmful"]["diff_expt_reason"]["diff_expt_reason"]["percent_diff_adduct"])
    diff_instrument.append(all_info_random[dataset][model]["diff_mol"]["harmful"]["diff_expt_reason"]["diff_expt_reason"]["percent_diff_instrument"])
    diff_adduct_CE.append(all_info_random[dataset][model]["diff_mol"]["harmful"]["diff_expt_reason"]["diff_expt_reason"]["percent_diff_adduct_CE"])
    diff_adduct_instrument.append(all_info_random[dataset][model]["diff_mol"]["harmful"]["diff_expt_reason"]["diff_expt_reason"]["percent_diff_adduct_instrument"])
    diff_instrument_CE.append(all_info_random[dataset][model]["diff_mol"]["harmful"]["diff_expt_reason"]["diff_expt_reason"]["percent_diff_instrument_CE"])
    diff_adduct_instrument_CE.append(all_info_random[dataset][model]["diff_mol"]["harmful"]["diff_expt_reason"]["diff_expt_reason"]["percent_diff_adduct_instrument_CE"])

diff_CE_bar = np.arange(len(diff_CE))
diff_adduct_bar = [x + bar_width + 0.01 for x in diff_CE_bar] 
diff_instrument_bar = [x + bar_width + 0.01 for x in diff_adduct_bar] 
diff_adduct_CE_bar = [x + bar_width + 0.01 for x in diff_instrument_bar] 
diff_adduct_instrument_bar = [x + bar_width + 0.01 for x in diff_adduct_CE_bar] 
diff_instrument_CE_bar = [x + bar_width + 0.01 for x in diff_adduct_instrument_bar] 
diff_adduct_instrument_CE_bar = [x + bar_width + 0.01 for x in diff_instrument_CE_bar] 

# plot bars
plt.bar(diff_CE_bar, diff_CE, width = bar_width, label = "% diff CE") #, color = "#D3D3D3")
plt.bar(diff_adduct_bar, diff_adduct, width = bar_width, label = "% diff adduct") #, color = "#F6BD60")
plt.bar(diff_instrument_bar, diff_instrument, width = bar_width, label = "% diff instrument") #, color = "#F6BD60")
plt.bar(diff_adduct_CE_bar, diff_adduct_CE, width = bar_width, label = "% diff adduct + CE") #, color = "#F6BD60")
plt.bar(diff_adduct_instrument_bar, diff_adduct_instrument, width = bar_width, label = "% diff adduct + instrument") #, color = "#F6BD60")
plt.bar(diff_instrument_CE_bar, diff_instrument_CE, width = bar_width, label = "% diff instrument + CE") #, color = "#F6BD60")
plt.bar(diff_adduct_instrument_CE_bar, diff_adduct_instrument_CE, width = bar_width, label = "% diff adduct + instrument + CE") #, color = "#F6BD60")

if dataset == "canopus":
    plt.title(f"Canopus", fontsize = 14)
if dataset == "massspecgym":
    plt.title(f"MassSpecGym (Harmful train samples)", fontsize = 14)

plt.xlabel('Models', fontsize = 12)
plt.ylabel('Percentage train samples (%)', fontsize = 12)
plt.ylim(bottom = 0, top = 100)
plt.xticks([r + bar_width * 4 for r in range(len(models))], ['Binned MLP', 'MS Transformer', 'Formula Transformer', "MIST"])

# for i in range(len(helpful)):
#     plt.text(helpful_bar[i], helpful[i], str(round(helpful[i], 1)) + "%", ha='center', va='bottom', fontsize = 8)
#     plt.text(harmful_bar[i], harmful[i], str(round(harmful[i], 1)) + "%", ha='center', va='bottom', fontsize = 8)

plt.legend()

In [ ]:
all_info_random[dataset]["MSG_formula_4096_random"]["diff_mol"]["helpful"]["diff_expt_reason"]["diff_expt_reason"]